In [5]:
import os
import shutil
from pathlib import Path
from ipywidgets import interact

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from core.Case import *
from core.globals import *
from core.Log import *


from methods.plots import *
from methods.preprocessing import *
from methods.intensity import *
from methods.slicing import *


CNTRL_dicom_root = "../Takotsubo-Syndrome/data/Inputs/normal_cases/"
TTS_dicom_root = "../Takotsubo-Syndrome/data/Inputs/takotsubo_cases/"
root_dir = "data/cases/"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
root_dir = Path("data/cases")
stats_list = []

loop=0
for caseID in os.listdir(root_dir):
	if caseID.startswith('.'):
		continue
	case = Case(caseID)
	set_log(caseID)
	casepath = root_dir / caseID
	infopath = casepath / "info.txt"
	#print(f"Processing {caseID}")
	## uncomment when info.txt content is fixed
	if infopath.exists():
		os.remove(infopath)
	try:
		stats = intensity_stats(case)
		stats["caseID"] = caseID
		stats_list.append(stats)
		#loop += 1
		#if loop > 5:
		#	break
	except Exception as e:
		print(f"Error processing: {e}")



In [2]:
ctrlID = "CNTRL_BJB_42898_78F"
ttsID = "TTS_JET_40340663_84F"
ctrlcase = Case(ctrlID)
ttscase = Case(ttsID)
scrollable_ct_mask_compare(ctrlcase, ttscase)


interactive(children=(IntSlider(value=31, description='slice_index', max=63), Output()), _dom_classes=('widget…

In [11]:
root_dir = Path("data/cases")
stats_list = []

loop=0
cases = os.listdir(root_dir)

spacings = []
for caseID in cases:
	if caseID.startswith('.'):
		continue
	#print(f"Processing {caseID}")
	case = Case(caseID)
	ct = case.croppedCT
	if ct is None:
		continue
	spacings.append(ct.spacing)  # tuple (sx, sy, sz)
spacings = np.array(spacings)
print("Mean spacing:", spacings.mean(axis=0))
print("Median spacing:", np.median(spacings, axis=0))
print("Range:", spacings.min(axis=0), spacings.max(axis=0))


Mean spacing: [0.71830285 0.7461681  1.7111081 ]
Median spacing: [0.703125 0.703125 1.25    ]
Range: [0.26171875 0.26171875 0.4       ] [0.976562  2.9999886 5.       ]


## Intensities:

In [ ]:
intensity_df = pd.DataFrame(stats_list)
plot_intensity_distribution(intensity_df)
